In [1]:
from __future__ import print_function
import argparse
import torchvision.datasets as dset
import torchvision.transforms as transforms
from PIL import Image
import json
from pprint import pprint

import matplotlib.pyplot as plt

from torch.nn import Parameter

import sys
import torch
import torch.nn as nn
from datetime import time
import numpy as np

import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import save_image

from torchvision import datasets, transforms
from torch.autograd import Variable



In [2]:
batch_size = 4
lr = 3E-4
momentum = 0.9

epochs = 30

cuda = False

log_interval = 20


In [3]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
from PIL import Image
import json
from pprint import pprint


def gs_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:

            return img.convert('L')

def load_data(parent_dir,transform = transforms.ToTensor()):
    return dset.ImageFolder(root=parent_dir, transform = transform, loader=gs_loader)


def load_target_mapping(filename = "../mapping.json"):
    with open(filename) as data_file:
        data = json.load(data_file)
        #collapse authors
        author = {}
        authors = []
        invertedMap = {}
        for object in data:
            a_name = object["Author"]
            f_name = object["Folder Name"]
            vol_name = object["Volume in dataset"]
            invertedMap[f_name] = a_name
            if vol_name is not None:
                invertedMap[f_name+"_vol{0:02d}".format(vol_name)] = a_name
            if not author.has_key(a_name) :
                author[a_name] = []
                authors.append(a_name)
            author[a_name].append(f_name)
            invertedMap[f_name] = a_name

        return  author, invertedMap, authors







In [4]:
train_loader = torch.utils.data.DataLoader(
    load_data("../../Data/Manga109_processed/images"),
    batch_size=batch_size, shuffle=True)

In [9]:
class convclassifier(nn.Module):
    def __init__(self, input_shape=(1, 256, 256),out_targets = 93):
        super(convclassifier, self).__init__()


        #encoder part
        self.en_conv_1 = nn.Conv2d(1, 100, 5)
        self.en_max_pool_1 =  nn.MaxPool2d(2,return_indices=False)  # b, 16, 5, 5
        self.en_conv_2 = nn.Conv2d(100, 200, 5) # b, 8, 3, 3
        self.en_max_pool_2 = nn.MaxPool2d(2,return_indices=False)  # b, 8, 2, 2

        input_size = self._get_conv_output_size(input_shape)
        self.dense_400 = nn.Linear(input_size,400)
        self.dense_200 = nn.Linear(400, 200)
        self.dense_out = nn.Linear(200, out_targets)

    def load_my_state_dict(self, state_dict):

        own_state = self.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                continue
            if isinstance(param, Parameter):
                # backwards compatibility for serialized parameters
                param = param.data
                print(param.size())
            print(param.size())
            print (own_state[name].size())    
            own_state[name].copy_(param)
    def _get_conv_output_size(self, shape):
        bs = 32
        input = Variable(torch.rand(bs, *shape))
        output_feat = self.en_conv_1(input)
        output_feat = self.en_max_pool_1(output_feat)
        output_feat = self.en_conv_2(output_feat)
        output_feat = self.en_max_pool_2(output_feat)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size

    def forward(self, x):
        x = self.en_conv_1(x)
        x = self.en_max_pool_1(x)
        x = self.en_conv_2(x)
        x = self.en_max_pool_2(x)
        print(x.size())
        Generate_images(x)
        x = x.view(x.size(0), -1)

        x = self.dense_400(x)
        x = self.dense_200(x)
        x = self.dense_out(x)


        return F.log_softmax(x)
def Generate_images(data):
    print ("Showing images")
    image_data  = to_img(data).numpy()
    
    plt.imshow(image_data[0:20])
    print ("Calling show images")
    plt.show()
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0)*200, 1, 61, 61)
    return x
def load_pretrained():
    pretrained_dict =  torch.load("/media/fahadm/Media/Trained Models/100/conv_autoencoder.pth", map_location=lambda storage, loc: storage)
    model.load_my_state_dict(pretrained_dict)


In [10]:
mapping, inveretedMap, authors = load_target_mapping();


model = convclassifier(out_targets=len(mapping))
criterion = nn.NLLLoss()

load_pretrained()


torch.Size([100, 1, 5, 5])
torch.Size([100, 1, 5, 5])
torch.Size([100])
torch.Size([100])
torch.Size([200, 100, 5, 5])
torch.Size([200, 100, 5, 5])
torch.Size([200])
torch.Size([200])
torch.Size([400, 744200])
torch.Size([400, 744200])
torch.Size([400])
torch.Size([400])
torch.Size([200, 400])
torch.Size([200, 400])
torch.Size([200])
torch.Size([200])
torch.Size([93, 200])
torch.Size([93, 200])
torch.Size([93])
torch.Size([93])


In [11]:
def map_target_to_author(target, loader ):
    arr = []
    for x in target.numpy():
        arr.append( authors.index( inveretedMap[loader.dataset.classes[x]]) )

    return torch.from_numpy(np.asarray(arr))
def test_classifier(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        old_target = target.numpy()
        target = map_target_to_author(target,train_loader)
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0]  # sum up batch loss
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

        print (correct/batch_size*batch_idx)

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


est_classifier(0)


test_classifier(0)

In [12]:
test_classifier(0)

torch.Size([4, 200, 61, 61])
Showing images


AttributeError: numpy

# 